In [ ]:
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from visualization_helper import *
from bending_validation import suppress_stdout as so

In [ ]:
import numpy as np
import numpy.linalg as la

In [ ]:
import importlib, pipeline_helper
importlib.reload(pipeline_helper)
from pipeline_helper import allEnergies, allGradientNorms, allDesignObjectives, allDesignGradientNorms

In [ ]:
input_path = '../../data/sphere_cap_0.3.json'
from load_jsondata import read_data
input_data, io = read_data(filepath = input_path)
import mesh
target_mesh = mesh.Mesh(input_data['target_v'], input_data['target_f'])
target_height_multiplier = input_data['target_spacing_factor']

### Initialization

In [ ]:
curr_um = umbrella_mesh.UmbrellaMesh(io)
thickness = io.material_params[6]
curr_um.targetDeploymentHeight = thickness

In [ ]:
curr_um.energy()

### Visualization

In [ ]:
rod_colors = get_color_field(curr_um, input_data)

In [ ]:
import mesh
view = linkage_vis.LinkageViewerWithSurface(curr_um, target_mesh, width=1024, height=600)
set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
view.showScalarField(rod_colors)
view.show()

#### Pin Rigid Motion



In [ ]:
use_pin = False

In [ ]:
driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-10
OPTS.verbose = 1
OPTS.beta = 1e-8
OPTS.niter = 1000
OPTS.verboseNonPosDef = False

In [ ]:
curr_um.deploymentForceType = umbrella_mesh.DeploymentForceType.LinearActuator

In [ ]:
import importlib, pipeline_helper
importlib.reload(pipeline_helper)

In [ ]:
from pipeline_helper import set_joint_vector_field, show_center_joint_normal, show_joint_normal

In [ ]:
from equilibrium_solve_analysis import EquilibriumSolveAnalysis
eqays = EquilibriumSolveAnalysis(curr_um)
def eqm_callback(prob, i):
    eqays.record(prob)
    if (i % 2 == 0):
        view.showScalarField(rod_colors)

In [ ]:
curr_um.targetDeploymentHeight = thickness * target_height_multiplier
curr_um.repulsionEnergyWeight = 0
curr_um.attractionWeight = 1e-1
curr_um.setHoldClosestPointsFixed(True)
curr_um.scaleInputPosWeights(1)

In [ ]:
curr_um.angleBoundEnforcement = umbrella_mesh.AngleBoundEnforcement.Hard

In [ ]:
allGradientNorms(curr_um)

In [ ]:
dof = curr_um.getDoFs()
for i in range(curr_um.numJoints()):
    # if (curr_um.joint(i).jointType() == umbrella_mesh.JointType.X):
    dof[curr_um.dofOffsetForJoint(i) + 6] = 1e-3
curr_um.setDoFs(dof)

In [ ]:
view.showScalarField(rod_colors)

In [ ]:
# curr_um.uniformDeploymentEnergyWeight = 1e-3
# with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e-2
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e-1
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
curr_um.uniformDeploymentEnergyWeight = 1e0
with so(): results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
results.success

In [ ]:
eqays.plot()

### Initialize Design Optimization

In [ ]:
import umbrella_optimization
import umbrella_optimization_finite_diff

In [ ]:
EO = umbrella_optimization.ElasticEnergyObjective(curr_um)

In [ ]:
curr_um.setHoldClosestPointsFixed(True)

In [ ]:
optimizer = umbrella_optimization.UmbrellaOptimization(curr_um, OPTS, 2.5, -1, False, fixedVars)

In [ ]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Elastic)

In [ ]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Target)

In [ ]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.DeploymentForce)

In [ ]:
optimizer.J(curr_um.getDesignParameters(), umbrella_optimization.OptEnergyType.Full)

In [ ]:
four_parameter_optimizer = umbrella_optimization.UmbrellaFourParametersOptimization(optimizer)

In [ ]:
len(four_parameter_optimizer.params()) / 4

In [ ]:
four_parameter_optimizer.params()

### Gradient Finite Difference Error

In [ ]:
direction = np.random.uniform(0, 1, four_parameter_optimizer.numParams())

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(four_parameter_optimizer, direction, umbrella_optimization.OptEnergyType.Full, minStepSize=1e-9, maxStepSize=1e-1)

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(four_parameter_optimizer, direction, umbrella_optimization.OptEnergyType.Elastic, minStepSize=1e-9, maxStepSize=1e-1)

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(four_parameter_optimizer, direction, umbrella_optimization.OptEnergyType.Target, minStepSize=1e-9, maxStepSize=1e0)

In [ ]:
umbrella_optimization_finite_diff.gradient_convergence_plot(four_parameter_optimizer, direction, umbrella_optimization.OptEnergyType.DeploymentForce, minStepSize=1e-9, maxStepSize=1e0)

### Hessian Finite Difference Errors

In [ ]:
umbrella_optimization_finite_diff.hessian_convergence_plot(four_parameter_optimizer, direction, umbrella_optimization_finite_diff.OptEnergyType.Full, minStepSize=1e-9, maxStepSize=1e1)

In [ ]:
error = umbrella_optimization_finite_diff.hessian_convergence_plot(four_parameter_optimizer, direction, umbrella_optimization.OptEnergyType.Elastic, minStepSize=1e-9, maxStepSize=1e1)

In [ ]:
umbrella_optimization_finite_diff.hessian_convergence_plot(four_parameter_optimizer, direction, umbrella_optimization.OptEnergyType.Target, maxStepSize=1e1, minStepSize=1e-9)

In [ ]:
umbrella_optimization_finite_diff.hessian_convergence_plot(four_parameter_optimizer, direction, umbrella_optimization.OptEnergyType.DeploymentForce, maxStepSize=1e1, minStepSize=1e-9)

In [ ]:
import importlib
importlib.reload(umbrella_optimization_finite_diff)